In [57]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [58]:
#定义超参数
input_size = 28  #图像尺寸
num_class = 10  #分类种数
num_epochs = 3  #训练总循环周期
batch_size = 64  #一批数据大小，64张图
#训练集
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
#测试集
test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor())

In [59]:
#构造batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

- 一搬卷积层，relu层，池化层可以写成一个组合
- 注意卷积结果还是一个特征图，需要转换成向量才可以做分类或者回归任务

In [60]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  #输入的图像 1*28*28 1代表颜色通道，这里用灰度图所以是1
            nn.Conv2d(  #图片，所以是2d
                in_channels=1,  #灰度图
                out_channels=16,  #要得到的特征图的个数，也代表卷积核的个数 输出特征为 (16,28,28)即16个28*28
                kernel_size=5,  #卷积核大小，
                stride=1,  #卷积移动步长
                padding=2,  # 如果希望卷积后大小跟原来一样，需要设置padding=(kernel_size-1)/2 if stride=1， 通常卷积核大小和padding设置 5->2,3,->1
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),  #池化层操作，2代表减小一半 输出为(16,14,14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 5, 1, 2),  #输入(16,14,14) 输出(32,14,14)
            nn.ReLU(),
            nn.Conv2d(32, 32, 5, 1, 2),  #输入(32,14,14)
            nn.ReLU(),
            nn.MaxPool2d(2),  #输出(32,7,7)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 5, 1, 2),  #输出(64,7,7)
            nn.ReLU(),
        )
        self.out = nn.Linear(64 * 7 * 7, 10)  #通过全连接成，转换为wx+b所需要的权重格式，才能对特征图进行预测或分类

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)  # flatter操作，得到一个视图，原来size是batch_size,64,64,7,结果为batch_size,64*7*7
        out = self.out(x)
        return out

In [61]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, dim=1)[1]  #[0]为值，【1】为索引，等同于数字0-9  dim 在每行的所有列中找最大值，即多个样本预测最高分的
    right = pred.eq(labels.data.view_as(pred)).sum() #view_as(tensor)：将当前张量的形状调整为与 tensor 相同。
    return right, len(labels)  #计算准确率使用

In [ ]:
#训练网络模型
net = CNN()
criterion = nn.CrossEntropyLoss()  #损失函数
#优化器
opt = optim.Adam(net.parameters(), lr=0.001)  #定义优化器,普通的随机梯度下降算法
# 开始训练
for epoch in range(num_epochs): #损失函数的优化是一个迭代过程，需要多次遍历数据以逼近最优解。例如，随机梯度下降（SGD）需要多轮更新才能收敛。
    train_rights = []
    for batch_idx, (data, target) in enumerate(train_loader):  #针对容器中的每一个批次进行循环
        net.train()
        output = net(data)
        loss = criterion(output, target)
        opt.zero_grad()
        loss.backward()
        opt.step()
        right = accuracy(output, target)
        train_rights.append(right)

        if batch_idx % 100 == 0:
            net.eval()
            val_rights = []

            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)

            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data,
                100. * train_r[0].numpy() / train_r[1],
                100. * val_r[0].numpy() / val_r[1]))


当前epoch: 0 [0/60000 (0%)]	损失: 2.304124	训练集准确率: 6.25%	测试集正确率: 10.00%
当前epoch: 0 [6400/60000 (11%)]	损失: 0.529631	训练集准确率: 77.95%	测试集正确率: 93.04%
当前epoch: 0 [12800/60000 (21%)]	损失: 0.175462	训练集准确率: 86.33%	测试集正确率: 96.78%
当前epoch: 0 [19200/60000 (32%)]	损失: 0.051850	训练集准确率: 89.61%	测试集正确率: 97.42%
当前epoch: 0 [25600/60000 (43%)]	损失: 0.085572	训练集准确率: 91.50%	测试集正确率: 97.91%
当前epoch: 0 [32000/60000 (53%)]	损失: 0.089012	训练集准确率: 92.67%	测试集正确率: 96.96%
当前epoch: 0 [38400/60000 (64%)]	损失: 0.007912	训练集准确率: 93.49%	测试集正确率: 98.17%
当前epoch: 0 [44800/60000 (75%)]	损失: 0.030787	训练集准确率: 94.07%	测试集正确率: 98.41%
当前epoch: 0 [51200/60000 (85%)]	损失: 0.017662	训练集准确率: 94.58%	测试集正确率: 98.55%
当前epoch: 0 [57600/60000 (96%)]	损失: 0.054747	训练集准确率: 94.98%	测试集正确率: 98.43%
当前epoch: 1 [0/60000 (0%)]	损失: 0.011451	训练集准确率: 100.00%	测试集正确率: 98.38%
当前epoch: 1 [6400/60000 (11%)]	损失: 0.017642	训练集准确率: 98.50%	测试集正确率: 98.50%
当前epoch: 1 [12800/60000 (21%)]	损失: 0.013514	训练集准确率: 98.52%	测试集正确率: 98.92%
当前epoch: 1 [19200/60000 (32%)]	损失: 0.009455	训练集准确率